# Assignment 3 : Segmenting and Clustering Neighborhoods in Toronto

First, it is required to load required libraries 

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup # library to pull data out of HTML and XML files
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # import k-means from clustering stage

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Download the HTML code of the web page and use the soup function to find the table. Then the function read from the pandas library is used to get the table as a dataframe

In [2]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
CanadaNeigh_df = pd.read_html(str(table))[0]
CanadaNeigh_df.head()

Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

The drop function is used to erase the rows where the Borough is not assigned. The loc function is used to replace the Neighborhoods that are not assigned with the corresponding borough name. Then, the index numbers are reset.

In [3]:
CanadaNeigh_df = CanadaNeigh_df.drop(CanadaNeigh_df[CanadaNeigh_df['Borough'] == 'Not assigned'].index)
CanadaNeigh_df.loc[(CanadaNeigh_df.Neighbourhood == 'Not assigned'),'Neighbourhood']=CanadaNeigh_df['Borough']
CanadaNeigh_df.reset_index(drop=True, inplace=True)
CanadaNeigh_df.head(11)

Postal Code           Borough                                Neighbourhood
0          M3A        North York                                    Parkwoods
1          M4A        North York                             Victoria Village
2          M5A  Downtown Toronto                    Regent Park, Harbourfront
3          M6A        North York             Lawrence Manor, Lawrence Heights
4          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
5          M9A         Etobicoke      Islington Avenue, Humber Valley Village
6          M1B       Scarborough                               Malvern, Rouge
7          M3B        North York                                    Don Mills
8          M4B         East York              Parkview Hill, Woodbine Gardens
9          M5B  Downtown Toronto                     Garden District, Ryerson
10         M6B        North York                                    Glencairn

The shape function is used to determine the number of rows and columns of the dataframe  

In [4]:
print('The dataframe of the neighbourhoods in Canada has {} rows and {} columns.'.format(CanadaNeigh_df.shape[0], CanadaNeigh_df.shape[1]))

The dataframe of the neighbourhoods in Canada has 103 rows and 3 columns.


The csv file with the geospatial positions is downloaded and saved as a dataframe

In [5]:
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data
Geospatial_df = pd.read_csv('Geospatial_data.csv')
Geospatial_df.head()

--2021-01-07 14:27:40--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2021-01-07 14:27:40--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-01-07 14:27:41--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-01-07 14:27:42--  https://ibm.box.com/public/static/9afzr83p

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

The dataframes with the Canada boroughs and the geospatial positions are merged, having into account that both dataframes have the Postal code column in common.

In [6]:
df = pd.merge(CanadaNeigh_df, Geospatial_df)
df.head(11)

Postal Code           Borough                                Neighbourhood  \
0          M3A        North York                                    Parkwoods   
1          M4A        North York                             Victoria Village   
2          M5A  Downtown Toronto                    Regent Park, Harbourfront   
3          M6A        North York             Lawrence Manor, Lawrence Heights   
4          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
5          M9A         Etobicoke      Islington Avenue, Humber Valley Village   
6          M1B       Scarborough                               Malvern, Rouge   
7          M3B        North York                                    Don Mills   
8          M4B         East York              Parkview Hill, Woodbine Gardens   
9          M5B  Downtown Toronto                     Garden District, Ryerson   
10         M6B        North York                                    Glencairn   

     Latitude  Longitude  
0   43.753259 -79.329656  
1   43.725882 -79.315572  
2   43.654260 -79.360636  
3   43.718518 -79.464763  
4   43.662301 -79.389494  
5   43.667856 -79.532242  
6   43.806686 -79.194353  
7   43.745906 -79.352188  
8   43.706397 -79.309937  
9   43.657162 -79.378937  
10  43.709577 -79.445073

The rows in which the word "Toronto" is not contained in the Borough column, are eliminated

In [7]:
df = df[df["Borough"].str.contains('Toronto', na=False)]
df.head(11)

Postal Code           Borough  \
2          M5A  Downtown Toronto   
4          M7A  Downtown Toronto   
9          M5B  Downtown Toronto   
15         M5C  Downtown Toronto   
19         M4E      East Toronto   
20         M5E  Downtown Toronto   
24         M5G  Downtown Toronto   
25         M6G  Downtown Toronto   
30         M5H  Downtown Toronto   
31         M6H      West Toronto   
36         M5J  Downtown Toronto   

                                        Neighbourhood   Latitude  Longitude  
2                           Regent Park, Harbourfront  43.654260 -79.360636  
4         Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
9                            Garden District, Ryerson  43.657162 -79.378937  
15                                     St. James Town  43.651494 -79.375418  
19                                        The Beaches  43.676357 -79.293031  
20                                        Berczy Park  43.644771 -79.373306  
24                                 Central Bay Street  43.657952 -79.387383  
25                                           Christie  43.669542 -79.422564  
30                           Richmond, Adelaide, King  43.650571 -79.384568  
31                       Dufferin, Dovercourt Village  43.669005 -79.442259  
36  Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752

The geopy library is used to get the latitude and longitude values of Toronto

In [8]:
address = 'Toronto, M'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7263287, -79.4518396.


The map of Toronto with neighborhoods superimposed on top is created.

In [9]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In order to explore the neighborhoods and segment them Foursquare API will be used. First, it is needed to define the Foursquare credentials and version

In [10]:
import os 

CLIENT_ID = os.environ.get('Foursquare_API_CLIENT_ID') # Foursquare ID
CLIENT_SECRET = os.environ.get('Foursquare_API_CLIENT_SECRET') # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('credentails uploaded')

credentails uploaded


The function to explore the venues of the neighborhoods in Toronto is defined

In [11]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

A limit of 100 venues in a radius of 500 meters is selected for each neighborhood

In [12]:
LIMIT = 100 
radius = 500 

The function above is used to get the venues of each Neighborhood in Toronto

In [13]:
toronto_venues = getNearbyVenues(names = df['Neighbourhood'],
                                 latitudes = df['Latitude'],
                                 longitudes = df['Longitude']
                                 ) 

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

This code shows the dimensions and the first rows of the dataframe obtained

In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(1618, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3      Morning Glory Cafe       43.653947       -79.361149   
4     Body Blitz Spa East       43.654735       -79.359874   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2  Distribution Center  
3       Breakfast Spot  
4                  Spa

the following code line shows how many venues were returned for each neighborhood

In [15]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            59   
Brockton, Parkdale Village, Exhibition Place                           22   
Business reply mail Processing Centre, South Ce...                     16   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     63   
Christie                                                               15   
Church and Wellesley                                                   80   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             35   
Davisville North                                                        9   
Dufferin, Dovercourt Village                                           15   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          24   
India Bazaar, The Beaches West                                         20   
Kensington Market, Chinatown, Grange Park                              65   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               43   
Moore Park, Summerhill East                                             3   
North Toronto West, Lawrence Park                                      18   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            35   
Regent Park, Harbourfront                                              45   
Richmond, Adelaide, King                                               95   
Rosedale                                                                4   
Roselawn                                                                4   
Runnymede, Swansea                                                     38   
St. James Town                                                         80   
St. James Town, Cabbagetown                                            44   
Stn A PO Boxes                                                         98   
Studio District                                                        37   
Summerhill West, Rathnelly, South Hill, Forest ...                     14   
The Annex, North Midtown, Yorkville                                    19   
The Beaches                                                             4   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         33   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             59   
Brockton, Parkdale Village, Exhibition Place                            22   
Business reply mail Processing Centre, South Ce...                      16   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      63   
Christie                                                                15   
Church and Wellesley                                                    80   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              35   
Davisville North                    

The dummie function is used to create a binary column for each category, then to this dataframe is inserted the column with the neighborhoods.

In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_neighbors = pd.DataFrame(toronto_venues['Neighborhood'])
toronto_onehotm = toronto_neighbors.merge(toronto_onehot, left_index=True, right_index=True)
toronto_onehotm = toronto_onehotm.rename(columns={'Neighborhood_x': 'Neighborhood'})
toronto_onehotm.drop(['Neighborhood_y'], axis=1, inplace=True)

print(toronto_onehotm.shape)
toronto_onehotm.head()

(1618, 235)


Neighborhood  Airport  Airport Food Court  Airport Lounge  \
0  Regent Park, Harbourfront        0                   0               0   
1  Regent Park, Harbourfront        0                   0               0   
2  Regent Park, Harbourfront        0                   0               0   
3  Regent Park, Harbourfront        0                   0               0   
4  Regent Park, Harbourfront        0                   0               0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0                0                 0                    0             0   
1                0                 0                    0             0   
2                0                 0                    0             0   
3                0                 0                    0             0   
4                0                 0                    0             0   

   Aquarium  Art Gallery  ...  Tibetan Restaurant  Toy / Game Store  Trail  \
0         0            0  ...                   0                 0      0   
1         0            0  ...                   0                 0      0   
2         0            0  ...                   0                 0      0   
3         0            0  ...                   0                 0      0   
4         0            0  ...                   0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Wine Shop  Yoga Studio  
0                      0         0          0            0  
1                      0         0          0            0  
2                      0         0          0            0  
3                      0         0          0            0  
4                      0         0          0            0  

[5 rows x 235 columns]

The rows are grouped by neighborhood and the mean frequency is taken for each category 

In [17]:
toronto_grouped = toronto_onehotm.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(39, 235)


Neighborhood  Airport  \
0                                         Berczy Park   0.0000   
1        Brockton, Parkdale Village, Exhibition Place   0.0000   
2   Business reply mail Processing Centre, South C...   0.0000   
3   CN Tower, King and Spadina, Railway Lands, Har...   0.0625   
4                                  Central Bay Street   0.0000   
5                                            Christie   0.0000   
6                                Church and Wellesley   0.0000   
7                      Commerce Court, Victoria Hotel   0.0000   
8                                          Davisville   0.0000   
9                                    Davisville North   0.0000   
10                       Dufferin, Dovercourt Village   0.0000   
11             First Canadian Place, Underground city   0.0000   
12    Forest Hill North & West, Forest Hill Road Park   0.0000   
13                           Garden District, Ryerson   0.0000   
14  Harbourfront East, Union Station, Toronto Islands   0.0000   
15                      High Park, The Junction South   0.0000   
16                     India Bazaar, The Beaches West   0.0000   
17          Kensington Market, Chinatown, Grange Park   0.0000   
18                                      Lawrence Park   0.0000   
19                           Little Portugal, Trinity   0.0000   
20                        Moore Park, Summerhill East   0.0000   
21                  North Toronto West, Lawrence Park   0.0000   
22                             Parkdale, Roncesvalles   0.0000   
23        Queen's Park, Ontario Provincial Government   0.0000   
24                          Regent Park, Harbourfront   0.0000   
25                           Richmond, Adelaide, King   0.0000   
26                                           Rosedale   0.0000   
27                                           Roselawn   0.0000   
28                                 Runnymede, Swansea   0.0000   
29                                     St. James Town   0.0000   
30                        St. James Town, Cabbagetown   0.0000   
31                                     Stn A PO Boxes   0.0000   
32                                    Studio District   0.0000   
33  Summerhill West, Rathnelly, South Hill, Forest...   0.0000   
34                The Annex, North Midtown, Yorkville   0.0000   
35                                        The Beaches   0.0000   
36                       The Danforth West, Riverdale   0.0000   
37           Toronto Dominion Centre, Design Exchange   0.0000   
38                     University of Toronto, Harbord   0.0000   

    Airport Food Court  Airport Lounge  Airport Service  Airport Terminal  \
0               0.0000           0.000            0.000             0.000   
1               0.0000           0.000            0.000             0.000   
2               0.0000           0.000            0.000             0.000   
3               0.0625           0.125            0.125             0.125   
4               0.0000           0.000            0.000             0.000   
5               0.0000           0.000            0.000             0.000   
6               0.0000           0.000            0.000             0.000   
7               0.0000           0.000            0.000             0.000   
8               0.0000           0.000            0.000             0.000   
9               0.0000           0.000            0.000             0.000   
10              0.0000           0.000            0.000             0.000   
11              0.0000           0.000            0.000             0.000   
12              0.0000           0.000            0.000             0.000   
13              0.0000           0.000            0.000             0.000   
14              0.0000           0.000            0.000             0.000   
15              0.0000           0.000            0.000             0.000   
16              0.0000           0.000            0.000             0.000   
17            

The next code is used to obtain the 5 most common categories in each neighborhood

In [18]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.05
2      Farmers Market  0.03
3              Bakery  0.03
4  Seafood Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3  Italian Restaurant  0.05
4             Stadium  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Garden Center  0.06
3          Comic Shop  0.06
4                Park  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2     Airport Terminal  0.12
3              Airport  0.06
4  Rental Car Location  0.06


----Central Bay Street----
      

This function is used to sort the venues in descending order

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

A dataframe is created to display the 10 most common venues categories in each neighborhood

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...    Light Rail Station   
3  CN Tower, King and Spadina, Railway Lands, Har...        Airport Lounge   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar              Beer Bar           Cheese Shop   
1        Breakfast Spot           Coffee Shop         Grocery Store   
2           Yoga Studio               Brewery         Garden Center   
3       Airport Service      Airport Terminal       Harbor / Marina   
4                  Café    Italian Restaurant        Sandwich Place   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                Bakery            Restaurant    Seafood Restaurant   
1            Restaurant               Stadium          Intersection   
2                Garden  Fast Food Restaurant        Farmers Market   
3      Sculpture Garden    Airport Food Court                   Bar   
4          Burger Joint           Salad Place       Bubble Tea Shop   

  8th Most Common Venue 9th Most Common Venue  10th Most Common Venue  
0        Farmers Market             Irish Pub                 Butcher  
1                   Bar                Bakery  Furniture / Home Store  
2            Comic Shop                  Park        Recording Studio  
3         Boat or Ferry              Boutique             Coffee Shop  
4   Japanese Restaurant       Thai Restaurant        Ramen Restaurant

The dataframe obtained above is used to cluster the neighborhoods in 6 groups

In [21]:
# set number of clusters
kclusters = 6
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 3, 0, 0, 2, 2, 2, 1, 0])

The dataframe with the cluster labels and most common venue category are grouped with the dataframe that contains the geospatial position of each neighborhood. Both dataframe have in common the neighborhood column, which is used to pair the rows. 

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() 

Postal Code           Borough                                Neighbourhood  \
2          M5A  Downtown Toronto                    Regent Park, Harbourfront   
4          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9          M5B  Downtown Toronto                     Garden District, Ryerson   
15         M5C  Downtown Toronto                               St. James Town   
19         M4E      East Toronto                                  The Beaches   

     Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
2   43.654260 -79.360636               0           Coffee Shop   
4   43.662301 -79.389494               0           Coffee Shop   
9   43.657162 -79.378937               0        Clothing Store   
15  43.651494 -79.375418               0           Coffee Shop   
19  43.676357 -79.293031               2                   Pub   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
2                 Bakery                  Café                  Park   
4       Sushi Restaurant                   Bar    Mexican Restaurant   
9            Coffee Shop                  Café        Cosmetics Shop   
15                  Café   American Restaurant          Cocktail Bar   
19     Health Food Store                 Trail           Yoga Studio   

   5th Most Common Venue      6th Most Common Venue 7th Most Common Venue  \
2         Breakfast Spot                    Theater                   Pub   
4                  Diner                   Beer Bar        Discount Store   
9                  Hotel  Middle Eastern Restaurant       Bubble Tea Shop   
15             Gastropub             Lingerie Store    Seafood Restaurant   
19                 Diner             Discount Store   Distribution Center   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
2                Brewery            Shoe Store            French Restaurant  
4     Italian Restaurant   Japanese Restaurant        Portuguese Restaurant  
9    Japanese Restaurant                 Diner               Lingerie Store  
15    Italian Restaurant            Restaurant                  Cheese Shop  
19               Dog Run      Doner Restaurant  Eastern European Restaurant

Now, we can see how the neighborhoods in each cluster are destributed in the map of Toronto 

In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In the table below, it is shown the amount of neighborhoods that there are in each cluster. The clusters 1, 4 and 5 only contain a single entry (neighborhood), therefore these could be taken as outliers that do not fit well in any cluster. In the map above it can be seen that the clusters 1, 3, 4 and 5 are all located mainly at the north of Toronto. On the contrary, the clusters 0 and 2 have 21 and 13 entries, respectively. In the map of Toronto (above), the neighborhoods belonging to the cluster 0 (red) are located in the center, while the neighborhoods of the cluster 2 (blue) are located on the surroundings.

In [24]:
toronto_merged.groupby('Cluster Labels').count()

Postal Code  Borough  Neighbourhood  Latitude  Longitude  \
Cluster Labels                                                             
0                        21       21             21        21         21   
1                         1        1              1         1          1   
2                        13       13             13        13         13   
3                         2        2              2         2          2   
4                         1        1              1         1          1   
5                         1        1              1         1          1   

                1st Most Common Venue  2nd Most Common Venue  \
Cluster Labels                                                 
0                                  21                     21   
1                                   1                      1   
2                                  13                     13   
3                                   2                      2   
4                                   1                      1   
5                                   1                      1   

                3rd Most Common Venue  4th Most Common Venue  \
Cluster Labels                                                 
0                                  21                     21   
1                                   1                      1   
2                                  13                     13   
3                                   2                      2   
4                                   1                      1   
5                                   1                      1   

                5th Most Common Venue  6th Most Common Venue  \
Cluster Labels                                                 
0                                  21                     21   
1                                   1                      1   
2                                  13                     13   
3                                   2                      2   
4                                   1                      1   
5                                   1                      1   

                7th Most Common Venue  8th Most Common Venue  \
Cluster Labels                                                 
0                                  21                     21   
1                                   1                      1   
2                                  13                     13   
3                                   2                      2   
4                                   1                      1   
5                                   1                      1   

                9th Most Common Venue  10th Most Common Venue  
Cluster Labels                                                 
0                                  21                      21  
1                                   1                       1  
2                                  13                      13  
3                                   2                       2  
4                                   1                       1  
5                                   1                       1

Now we will examine the categories of the venues in each cluster to determine what kind of clusters were obtained 

## Cluster 0

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
2   Downtown Toronto               0           Coffee Shop   
4   Downtown Toronto               0           Coffee Shop   
9   Downtown Toronto               0        Clothing Store   
15  Downtown Toronto               0           Coffee Shop   
20  Downtown Toronto               0           Coffee Shop   
24  Downtown Toronto               0           Coffee Shop   
30  Downtown Toronto               0           Coffee Shop   
36  Downtown Toronto               0           Coffee Shop   
37      West Toronto               0                   Bar   
41      East Toronto               0      Greek Restaurant   
42  Downtown Toronto               0           Coffee Shop   
48  Downtown Toronto               0           Coffee Shop   
54      East Toronto               0           Coffee Shop   
73   Central Toronto               0           Coffee Shop   
75      West Toronto               0        Breakfast Spot   
86   Central Toronto               0           Coffee Shop   
87  Downtown Toronto               0        Airport Lounge   
92  Downtown Toronto               0           Coffee Shop   
96  Downtown Toronto               0           Coffee Shop   
97  Downtown Toronto               0           Coffee Shop   
99  Downtown Toronto               0           Coffee Shop   

   2nd Most Common Venue 3rd Most Common Venue  4th Most Common Venue  \
2                 Bakery                  Café                   Park   
4       Sushi Restaurant                   Bar     Mexican Restaurant   
9            Coffee Shop                  Café         Cosmetics Shop   
15                  Café   American Restaurant           Cocktail Bar   
20          Cocktail Bar              Beer Bar            Cheese Shop   
24                  Café    Italian Restaurant         Sandwich Place   
30                  Café            Restaurant          Deli / Bodega   
36              Aquarium                  Café                  Hotel   
37           Coffee Shop            Restaurant  Vietnamese Restaurant   
41    Italian Restaurant           Coffee Shop              Bookstore   
42                 Hotel                  Café            Salad Place   
48            Restaurant                  Café                  Hotel   
54                  Café                Bakery                Brewery   
73        Clothing Store           Yoga Studio   Fast Food Restaurant   
75             Gift Shop         Movie Theater           Dessert Shop   
86           Pizza Place            Restaurant    Fried Chicken Joint   
87       Airport Service      Airport Terminal        Harbor / Marina   
92    Seafood Restaurant                 Hotel             Restaurant   
96                Bakery           Pizza Place                   Café   
97                  Café                 Hotel             Restaurant   
99   Japanese Restaurant      Sushi Restaurant             Restaurant   

            5th Most Common Venue      6th Most Common Venue  \
2                  Breakfast Spot                    Theater   
4                           Diner                   Beer Bar   
9                           Hotel  Middle Eastern Restaurant   
15                      Gastropub             Lingerie Store   
20                         Bakery                 Restaurant   
24                   Burger Joint                Salad Place   
30                          Hotel            Thai Restaurant   
36                        Brewery             Scenic Lookout   
37  Vegetarian / Vegan Restaurant                Men's Store   
41         Furniture / Home Store             Ice Cream Shop   
42             Seafood Restaurant                 Restaurant   
48                            Gym        American Restaurant   
54            American Restaurant                  Gastropub   
73             Chinese Restaurant                 Restaurant   
75    Eastern European Restaurant                    Dog Run   
86               

## Cluster 2

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
19       East Toronto               2                   Pub   
25   Downtown Toronto               2         Grocery Store   
31       West Toronto               2                Bakery   
43       West Toronto               2                  Café   
47       East Toronto               2  Fast Food Restaurant   
67    Central Toronto               2           Pizza Place   
69       West Toronto               2                  Café   
74    Central Toronto               2        Sandwich Place   
79    Central Toronto               2        Sandwich Place   
80   Downtown Toronto               2                  Café   
81       West Toronto               2      Sushi Restaurant   
84   Downtown Toronto               2                  Café   
100      East Toronto               2    Light Rail Station   

    2nd Most Common Venue  3rd Most Common Venue  \
19      Health Food Store                  Trail   
25                   Café                   Park   
31               Pharmacy          Grocery Store   
43         Breakfast Spot            Coffee Shop   
47            Coffee Shop                    Pub   
67         Breakfast Spot   Gym / Fitness Center   
69     Mexican Restaurant        Thai Restaurant   
74                   Café            Coffee Shop   
79           Dessert Shop       Sushi Restaurant   
80                 Bakery                    Bar   
81                   Café            Pizza Place   
84            Coffee Shop  Vietnamese Restaurant   
100           Yoga Studio                Brewery   

             4th Most Common Venue 5th Most Common Venue  \
19                     Yoga Studio                 Diner   
25                       Nightclub    Italian Restaurant   
31                       Pet Store                  Pool   
43                   Grocery Store            Restaurant   
47                   Burrito Place        Sandwich Place   
67                           Hotel     Food & Drink Shop   
69                            Park        Discount Store   
74                     Pizza Place                   Pub   
79                     Coffee Shop           Pizza Place   
80              Italian Restaurant   Japanese Restaurant   
81              Italian Restaurant                   Pub   
84   Vegetarian / Vegan Restaurant    Mexican Restaurant   
100                  Garden Center                Garden   

    6th Most Common Venue      7th Most Common Venue 8th Most Common Venue  \
19         Discount Store        Distribution Center               Dog Run   
25            Candy Store                 Baby Store            Restaurant   
31            Music Venue  Middle Eastern Restaurant                  Café   
43                Stadium               Intersection                   Bar   
47             Board Shop          Fish & Chips Shop    Italian Restaurant   
67       Department Store                       Park        Sandwich Place   
69    Fried Chicken Joint                      Diner                   Bar   
74                   Park                 Donut Shop          Burger Joint   
79        Thai Restaurant         Italian Restaurant                  Café   
80              Bookstore                        Pub              Beer Bar   
81            Coffee Shop         Falafel Restaurant            Restaurant   
84         Farmers Market                Gaming Cafe                   Bar   
100  Fast Food Restaurant             Farmers Market            Comic Shop   

         9th Most Common Venue       10th Most Common Venue  
19            Doner Restaurant  Eastern European Restaurant  
25                 Coffee Shop                     Creperie  
31                     Brewery                  Supermarket  
43                      Bakery       Furniture / Home Store  
47                  Steakhouse                   Restaurant  
67                         Gym                  Art Gallery  
69   Cajun / Creole Restaurant    

## Cluster 3

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
68   Central Toronto               3                  Park   
91  Downtown Toronto               3                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
68                 Trail         Jewelry Store      Sushi Restaurant   
91            Playground                 Trail           Yoga Studio   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
68           Yoga Studio                 Diner           Event Space   
91      Department Store  Ethiopian Restaurant           Escape Room   

   8th Most Common Venue        9th Most Common Venue 10th Most Common Venue  
68  Ethiopian Restaurant                  Escape Room      Electronics Store  
91     Electronics Store  Eastern European Restaurant             Donut Shop

## Cluster 1, 4 and 5

In [28]:
toronto_merged.loc[(toronto_merged['Cluster Labels'] == 1) | (toronto_merged['Cluster Labels'] == 4) | (toronto_merged['Cluster Labels'] == 5)]

Postal Code          Borough                Neighbourhood   Latitude  \
61         M4N  Central Toronto                Lawrence Park  43.728020   
62         M5N  Central Toronto                     Roselawn  43.711695   
83         M4T  Central Toronto  Moore Park, Summerhill East  43.689574   

    Longitude  Cluster Labels 1st Most Common Venue    2nd Most Common Venue  \
61 -79.388790               1                  Park                 Bus Line   
62 -79.416936               4                Garden  Health & Beauty Service   
83 -79.383160               5            Restaurant                    Trail   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
61           Swim School      Business Service           Yoga Studio   
62                  Pool          Home Service            Donut Shop   
83          Tennis Court      Department Store           Event Space   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
61        Discount Store           Event Space  Ethiopian Restaurant   
62                 Diner        Discount Store   Distribution Center   
83  Ethiopian Restaurant           Escape Room     Electronics Store   

          9th Most Common Venue 10th Most Common Venue  
61                  Escape Room      Electronics Store  
62                      Dog Run       Doner Restaurant  
83  Eastern European Restaurant             Donut Shop

Considering the cluster characteristics, 4 big groups were identified 
- Cluster 0: Cafes, bars and restaurants, located in the center
- Cluster 2: shops and restaurants, located surrounding the center
- Cluster 3: parks and other stores, located north
- Cluster 1,4,5: different venues, located north